In [1]:
from tools import (
    load_book, 
    format_text_with_line_breaks, 
    extract_keywords_from_fragments, 
    generate_wordcloud,
    initialize_vector_database,
    insert_text_fragments,
    hybrid_search,
    reciprocal_rank_fusion,
    answer_question_with_context_streaming,
    process_text_into_chunks
)
from text_chunking.SemanticClusterVisualizer import SemanticClusterVisualizer
import seaborn as sns
import logging
import warnings
import numpy as np
import pandas as pd
from IPython.display import clear_output, display, HTML
import markdown

logging.getLogger('langchain').setLevel(logging.ERROR)
logging.getLogger('requests').setLevel(logging.ERROR)
logging.getLogger('openai').setLevel(logging.ERROR)
logging.basicConfig(format="%(asctime)s - %(message)s", level=logging.ERROR)

warnings.filterwarnings("ignore", message=".*force_all_finite.*")
warnings.filterwarnings("ignore", message=".*n_jobs value.*overridden to 1 by setting random_state.*")
logging.basicConfig(format="%(asctime)s - %(message)s", level=logging.WARNING)

for module in ["sklearn", "umap"]:
    logging.getLogger(module).setLevel(logging.ERROR)


# DDL y preparación de Postgres DB

La base de datos PostgreSQL funciona como el almacén central de nuestro sistema. Imagina una biblioteca muy organizada donde guardamos tanto los textos completos como representaciones matemáticas de su significado.

En esta etapa:

- Configuramos PostgreSQL con una extensión especial llamada "pgvector" que le permite entender matemáticamente el significado de fragmentos de textos
- Creamos una tabla llamada "chunks" (fragmentos) para almacenar:
  * El texto original
  * Su representación matemática (vectores)
  * Un formato especial para búsquedas de palabras exactas
  * Información adicional como palabras clave

También implementamos índices, que son como los sistemas de clasificación en una biblioteca. Estos nos permiten encontrar rápidamente información sin tener que revisar todo el contenido, usando tanto el significado como las palabras exactas.

Es similar a preparar un terreno y construir los cimientos antes de levantar una casa: estamos creando la infraestructura básica donde todo lo demás se apoyará.

In [ ]:
connection = initialize_vector_database(
    host="localhost",
    port=5432,
    dbname="workshop_rag",
    user="postgres",
    password="dev.2m",
    vector_dimensions= 1024 # Nuestro modelo de embeddings tiene 1024 dimensiones (open AI tiene 1536)
)

# Ingesta de libro- Texto y pre-fragmentado

En esta fase, nuestro sistema "lee" el libro y lo prepara para un procesamiento más avanzado, como un chef que prepara los ingredientes antes de cocinar.

El proceso incluye:

- **Cargar el libro**: Leemos el archivo de texto completo, asegurándonos de que se interpretan correctamente los caracteres especiales del español (como acentos y eñes).

- **Fragmentación inicial del texto**: Organizamos el texto en párrafos simples de manera limpia y consistente, respetando la estructura natural del libro.

In [ ]:
full_text = load_book("./book.txt")
sns.set_context("talk")

semantic_chunker = SemanticClusterVisualizer(
    api_key= "123", 
    llm_model='gpt-4o',
    base_url_llm= "http://localhost:3000/v1",
    base_url_embeddings= "http://localhost:3001/v1",
    embeddings_model= "text-embedding-3-small"
)

# split the document into chunks
original_split_texts = process_text_into_chunks(
    full_text
)
print(f"Original split texts: {len(original_split_texts)}\n\n")
print(format_text_with_line_breaks("\n\n".join(original_split_texts[:10])))

# Pre-proceso para búsquedas semánticas

Aquí es donde el sistema aprende a "entender" el significado del texto, no solo las palabras exactas.

Imagina que traduces cada fragmento de texto a un idioma universal matemático. Este componente:

- **Crea "embeddings"**: Convierte texto en largas listas de números (vectores) donde textos con significados similares tendrán números similares, incluso si usan palabras diferentes.
  * Por ejemplo, "estoy feliz" y "me siento contento" tendrían representaciones numéricas parecidas aunque usen palabras distintas.

- **Realiza chunking semántico**: Este proceso es fundamental para preservar el significado:
  * Toma los fragmentos básicos generados en la etapa anterior
  * Analiza la similitud semántica entre fragmentos consecutivos usando sus embeddings
  * Agrupa fragmentos que están conceptualmente relacionados
  * Evita cortes en medio de explicaciones, definiciones o argumentos importantes
  * Prioriza mantener juntos párrafos que complementan una misma idea

- **Preserva contextos completos**: A diferencia del fragmentado simple por cantidad de caracteres o párrafos:
  * Identifica secciones temáticas completas
  * Evita separar elementos que dependen unos de otros para su comprensión

- **Equilibra tamaño y coherencia**: Busca un balance óptimo para que los fragmentos:
  * Sean lo suficientemente grandes para contener ideas completas
  * No tan extensos que diluyan el significado central
  * Mantengan relaciones causa-efecto dentro del mismo fragmento

Este proceso emula cómo los humanos entendemos textos: agrupamos naturalmente ideas relacionadas y mantenemos el hilo conductor de un argumento, incluso cuando abarca varios párrafos. El chunking semántico es crucial para que el sistema pueda ofrecer respuestas coherentes y completas, evitando información fragmentada o sacada de contexto.


In [ ]:
# run embeddings
original_split_text_embeddings = semantic_chunker.embed_original_document_splits(original_split_texts)

# generate breakpoints, use length threshold to decide which 
# sections to further subdivide 
breakpoints, semantic_groups = semantic_chunker.generate_breakpoints(
    original_split_texts,
    original_split_text_embeddings,
    percentile_threshold= 0.9,
    length_threshold= 5000,
    plot=True
)

print(f'breakpoints: {breakpoints}')
print(f'original_split_texts #: {len(original_split_texts)}')
print(f'semantic_groups #: {len(semantic_groups)}')
print(f'avg word count of semantic_groups: {np.mean([len(x.split()) for x in semantic_groups])}')
print(f'semantic_groups: {format_text_with_line_breaks(semantic_groups[2])}')

# embed the groups that have been made from the breakpoints
semantic_group_embeddings = semantic_chunker.embed_semantic_groups(semantic_groups)

# cluster the groups
splits_df, semantic_group_clusters = semantic_chunker.vizualize_semantic_groups(
    semantic_groups,
    semantic_group_embeddings,
    n_clusters= 8
)

# generate cluster summaries
cluster_summaries = semantic_chunker.generate_cluster_labels(
    semantic_group_clusters, plot= True
)

# generate cluster bounds
semantic_cluster_bounds = semantic_chunker.split_visualizer.plot_corpus_and_clusters(
    splits_df, cluster_summaries
)

# Pre-proceso para búsquedas por coincidencias de la información

Este componente se enfoca en las palabras más importantes de cada texto, como un estudiante que subraya los términos clave en sus apuntes.

El proceso incluye:

- **Extracción de palabras clave**: Usa un algoritmo llamado BM25 (similar al que usa Google) para identificar qué palabras son realmente importantes en cada fragmento.
  * Prioriza palabras que aparecen mucho en un fragmento específico pero poco en el resto del libro.
  * Por ejemplo, en un capítulo sobre "fotosíntesis", esta palabra sería muy relevante si apenas aparece en otros capítulos.

- **Procesamiento eficiente**: Distribuye este trabajo entre varios núcleos del procesador para hacerlo más rápido, como tener varios asistentes analizando diferentes párrafos simultáneamente.

- **Visualización de palabras clave**: Crea "nubes de palabras" donde los términos más importantes aparecen más grandes, ofreciendo una representación visual rápida del contenido.

Estas palabras clave complementan la búsqueda semántica, permitiendo encontrar fragmentos que contienen exactamente los términos que buscamos, no solo conceptos similares.

In [ ]:
# Extraer keywords de cada fragmento (10 por defecto)
key_words_for_semantic_groups = extract_keywords_from_fragments(semantic_groups, full_text, top_n= 30)

# Acceder a los keywords de un fragmento específico
keywords_primer_fragmento = key_words_for_semantic_groups[0]
generate_wordcloud(key_words_for_semantic_groups, "Palabras claves extraídas de los fragmentos de texto")

# Creación de índice híbrido para recuperación de información

En esta fase construimos un sistema de búsqueda dual que combina lo mejor de dos mundos: la búsqueda por significado (semántica) y la búsqueda por palabras exactas.

El proceso es como crear un índice doble para un libro:

- **Almacenamiento completo**: Para cada fragmento de texto guardamos:
  * El texto original completo
  * Su traducción matemática (vector de embeddings)
  * Las palabras clave importantes
  * Información adicional como cuándo fue procesado

- **Procesamiento optimizado**: Trabajamos con grupos de fragmentos a la vez para ser eficientes.
  * Es similar a enviar documentos a archivar en lotes, en lugar de uno por uno.
  * El sistema informa del progreso para saber cuánto falta.

- **Indexación automática**: La base de datos crea automáticamente sistemas de búsqueda rápida:
  * Uno para encontrar textos con significado similar (como un índice temático)
  * Otro para encontrar palabras específicas (como un índice alfabético)

Este componente es como un bibliotecario experto que puede encontrar libros tanto por su tema general como por palabras específicas que contienen.

In [ ]:
inserted_count = insert_text_fragments(
    connection= connection,
    text_fragments= semantic_groups,
    keywords_lists= key_words_for_semantic_groups,
    api_key= "123",
    base_url_embeddings= "http://localhost:3001/v1"
)

# Recuperación híbrida

Este es el componente que realmente realiza las búsquedas combinando dos enfoques potentes:

- **Búsqueda semántica** (por significado):
  * Convierte la pregunta del usuario en un vector matemático de embeddings
  * Encuentra fragmentos cuyo vector es similar, aunque usen palabras diferentes
  * Es como encontrar recetas similares aunque usen términos culinarios distintos

- **Búsqueda textual** (por palabras):
  * Extrae las palabras importantes de la pregunta
  * Busca fragmentos que contengan exactamente esas palabras
  * Es como buscar recetas que mencionen específicamente "chocolate" o "sin gluten"

El sistema combina inteligentemente ambos resultados:
  * Elimina duplicados (textos encontrados por ambos métodos)
  * Da preferencia a fragmentos encontrados tanto por significado como por palabras exactas
  * Ajusta la cantidad de resultados según lo que encuentre

Es como tener dos asistentes de investigación: uno experto en entender conceptos generales y otro especializado en encontrar términos específicos, trabajando juntos para darte la mejor respuesta.

In [ ]:
query = 'Tienes información sobre el experimento de la doble rendija'
results = hybrid_search(
    connection= connection,
    query_text= query,
    api_key= "123",
    base_url_embeddings= "http://localhost:3001/v1",
    top_k= 20
)
print(len(results))
pd.DataFrame(results)

# Componente de Ranking por relevancia (Reciprocal Rank Fusion)

Este componente decide qué fragmentos son realmente los más relevantes para la pregunta, como un juez que evalúa las respuestas de diferentes expertos.

El proceso, llamado Reciprocal Rank Fusion (RRF), funciona así:

- **Combina los rankings de ambos métodos de búsqueda**:
  * Da importancia a la posición en cada ranking (los primeros lugares valen más)
  * También considera cuán relevante era cada fragmento según cada método
  * Usa una fórmula matemática especial para que ningún método domine completamente

- **Premia la consistencia**:
  * Si un fragmento aparece en ambos métodos de búsqueda, recibe un 20% de puntuación extra
  * Esto refleja que probablemente es más relevante si dos métodos diferentes lo encontraron

- **Reordena los resultados**:
  * Crea una lista final ordenada por relevancia combinada
  * Mantiene solo los mejores resultados (top_k)

Este componente es como un director de orquesta que toma las contribuciones de diferentes músicos (métodos de búsqueda) y las armoniza en una sola pieza coherente, destacando las partes donde hay mayor acuerdo.

In [ ]:
reranked_results = reciprocal_rank_fusion(results, top_k= 7, k_constant= 60.0)
pd.DataFrame(reranked_results)

# Prompting

El "prompting" es como dar instrucciones precisas a un asistente inteligente para que responda preguntas usando solo la información proporcionada.

Este componente:

- **Crea instrucciones claras (prompts)** para el modelo de lenguaje:
  * Incluye la pregunta original del usuario
  * Proporciona los fragmentos relevantes encontrados, numerados como "Documento 1", "Documento 2", etc.
  * Da instrucciones específicas: "Responde solo usando la información proporcionada"

- **Formatea la información adecuadamente**:
  * Organiza el contexto de manera estructurada
  * Asegura que el modelo pueda distinguir claramente entre diferentes fragmentos

Este componente es como un traductor entre los resultados de la búsqueda y el modelo de lenguaje, asegurándose de que este último entienda exactamente qué información debe usar y cómo responder.

In [9]:
prompt = """Eres un asistente virtual especializado cuya función es proporcionar información basándote EXCLUSIVAMENTE 
en la base de conocimientos que se te ha proporcionado. Tu misión es:

1. Analizar cuidadosamente cada consulta del usuario.
2. Buscar información relevante ÚNICAMENTE dentro de la base de conocimientos proporcionada.
3. Responder de manera detallada y estructurada utilizando formato Markdown.
4. NO utilizar conocimientos propios, especulaciones o información externa que no esté explícitamente incluida en la base de conocimientos.
5. Estructurar tus respuestas con títulos, subtítulos, listas y énfasis cuando sea apropiado para mejorar la legibilidad.

## Casos especiales:

Si la base de conocimientos NO contiene información suficiente para responder a la consulta del usuario:
1. Indica claramente: "No dispongo de información suficiente en mi base de conocimientos para responder a esta consulta específica."
2. Ofrece información sobre un tema relacionado que SÍ esté presente en la base de conocimientos, introduciendo con: "Sin embargo, puedo ofrecerte información sobre [tema relacionado] que podría ser de tu interés:"

## Formato de respuesta:

Utiliza Markdown para estructurar tus respuestas:
- Usa `#` para títulos principales
- Usa `##` para subtítulos
- Utiliza listas con `-` o `1.`
- Emplea `**texto**` para enfatizar conceptos importantes
- Utiliza `>` para citas textuales de la base de conocimientos
- Implementa tablas cuando sea apropiado para presentar datos estructurados

Recuerda: 
- Tu valor reside en proporcionar ÚNICAMENTE información verificada que exista en tu base de conocimientos, sin añadir especulaciones ni conocimientos externos.

Solicitud del usuario: {query}

Base de conocimientos: {context}"""

# Generación de respuesta a través de LLM (Local)

Este componente final utiliza un Modelo de Lenguaje Grande (LLM) local para generar respuestas naturales basadas en la información recuperada.

El proceso funciona así:

- **Respuestas contextualizadas**:
  * El modelo usa específicamente los fragmentos relevantes encontrados
  * Mantiene la respuesta enfocada en la información proporcionada

- **Se adapta a modelos locales**:
  * Funciona con modelos de lenguaje ejecutados en el servidor local
  * Permite ajustar parámetros como la "temperatura" (qué tan creativo o conservador será el modelo)

Este componente es como un escritor experto que estudia los fragmentos seleccionados y redacta una respuesta coherente y natural, mostrándote su trabajo mientras lo hace.


In [ ]:
stream = answer_question_with_context_streaming(
    query= query, context_docs= results, api_key= "123",
    base_url= "http://localhost:3000/v1", model= "gpt-4o",
    prompt_template=prompt
)

output = ""
for token in stream:
    output += token
    clear_output(wait=True)
    
    # Convertir Markdown a HTML
    html_content = markdown.markdown(output)
    
    # Aplicar estilo al HTML
    styled_html = f"""
    <div style="font-size: 18px; line-height: 1.5;">
        {html_content}
    </div>
    """
    
    display(HTML(styled_html))